## FIT5120: IE Studio experience project
### Project: Link2 HerResilience

Team: Harmony (TA18)
Author: Mandeep Singh

Data wrangling process for epic6: yoga dataset

In [1]:
import os
import csv


In [2]:
path_dir = os.getcwd() + "/epic6/Yoga Postures Dataset"
path_images_dir = path_dir + "/Images"

list_yoga_asanas = [x.lower() for x in os.listdir(path_images_dir)]
list_yoga_asanas

['anantasana',
 'ardhakati chakrasana',
 'vajrasana',
 'tadasana',
 'marjariasana',
 'parvatasana',
 'bhujangasana',
 'viparita karani',
 'kati chakrasana',
 'sarvangasana']

### Pre-processing: Extract & transform

In [3]:
import re

dict_yoga_asanas_wrong_postures = {}
for asana in list_yoga_asanas:
    list_wrong_postures_in_sequence = os.listdir(path_images_dir + "/" + asana + "/" + asana + " Wrong Steps")
    
    # dict of wrong_postures in a step
    dict_wrong_postures_in_step = {}
    for s in list_wrong_postures_in_sequence:
        # extract asana step number from s
        step = int(re.search(r"Step (\d)", s)[1])
        
        # extract posture
        posture = re.search(r"\d- ?([A-Za-z ]*)", s)[1].lower()
        
        if not step in dict_wrong_postures_in_step.keys():
            dict_wrong_postures_in_step[step] = [posture]
        else:
            dict_wrong_postures_in_step[step].append(posture)
    
    dict_yoga_asanas_wrong_postures[asana] = dict_wrong_postures_in_step
    
dict_yoga_asanas_wrong_postures


{'anantasana': {2: ['knee'], 1: ['legs', 'neck'], 3: ['knee'], 4: ['hand']},
 'ardhakati chakrasana': {2: ['ankle', 'head'],
  3: ['knee', 'legs'],
  5: ['knee', 'legs'],
  4: ['legs'],
  1: ['head', 'palms']},
 'vajrasana': {2: ['back and head'], 1: ['legs and hand']},
 'tadasana': {1: ['legs'],
  2: ['legs and hand'],
  5: ['standing'],
  3: ['legs and hand'],
  4: ['hands and legs']},
 'marjariasana': {2: ['legs'],
  3: ['legs'],
  4: ['hand and legs'],
  1: ['hand and neck']},
 'parvatasana': {3: ['leg and hand'],
  2: ['leg and hand', 'hand'],
  1: ['leg and hand'],
  4: ['hand']},
 'bhujangasana': {2: ['legs', 'hand and legs'],
  1: ['head', 'legs'],
  3: ['legs', 'head']},
 'viparita karani': {1: ['legs', 'neck'],
  2: ['legs', 'knees', 'neck', 'back ']},
 'kati chakrasana': {2: ['hand'], 4: ['neck'], 1: ['legs'], 3: ['neck']},
 'sarvangasana': {1: ['neck and hand', 'neck', 'legs'],
  2: ['legs', 'back ', 'knees']}}

In [4]:
# add 2 more as per functionality
dict_yoga_asanas_wrong_postures['Pranayama'] = {1: ['Legs', 'Hand'], 2: ['Hand'], 3: ['Hand']}
dict_yoga_asanas_wrong_postures['Chakrasana'] = {1: ['Legs', 'Back'], 2: ['Legs', 'Back', 'Hand'], 
                                                 3: ['Legs', 'Back', 'Hips'], 4: ['Legs', 'Back']}


In [5]:
import json

# save in json
with open(path_dir + "/asanas_wrong_postures_in_step.json", "w") as file:
    json.dump(dict_yoga_asanas_wrong_postures, file)

